In [54]:
#Import packages needed
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import requests
import gmaps
from config import gkey
from pprint import pprint
gmaps.configure(api_key=gkey)

## Bringing in the data

In [2]:
#Bringing in cime data and making it a dataframe
crimes_csv = "LACrimes2013Zip.csv"
crimes_df = pd.read_csv(crimes_csv)
crimes_df.head()

,Date.Rptd,DR.NO,DATE.OCC,TIME.OCC,AREA,AREA.NAME,RD,Crm.Cd,CrmCd.Desc,Status,Status.Desc,LOCATION,Cross.Street,Location.1,Formatted Address,Zipcode
0,03/20/2013,132007717,03/20/2013,2015,20,Olympic,2004,997,TRAFFIC DR #,UNK,Unknown,OXFORD,OAKWOOD,"(34.0776, -118.308)","4650-4652 Oakwood Ave, Los Angeles, CA 90004, USA",90004
1,03/10/2013,130608787,03/10/2013,445,6,Hollywood,635,997,TRAFFIC DR #,UNK,Unknown,ODIN ST,CAHUENGA BL,"(34.1113, -118.3336)","2314 N Cahuenga Blvd, Los Angeles, CA 90068, USA",90068
2,12/18/2013,131820260,12/18/2013,745,18,Southeast,1839,997,TRAFFIC DR #,UNK,Unknown,105TH ST,CROESUS AV,"(33.9406, -118.2338)","2148 E 105th St, Los Angeles, CA 90002, USA",90002
3,10/18/2013,131817514,10/18/2013,1730,18,Southeast,1827,997,TRAFFIC DR #,UNK,Unknown,101ST ST,JUNIPER ST,"(33.9449, -118.2332)","10100-10198 Juniper St, Los Angeles, CA 90002,...",90002
4,05/26/2013,130510483,05/25/2013,2000,5,Harbor,507,440,THEFT PLAIN - PETTY (UNDER $400),UNK,Unknown,1300 W SEPULVEDA BL,NaN,"(33.8135, -118.2992)","1303 Sepulveda Blvd, Torrance, CA 90501, USA",90501


In [3]:
#Bringing in bike data and making it a dataframe
bike_csv = "los-angeles-metro-bike-share-trip-data/metro-bike-share-trip-data.csv"
bike_df = pd.read_csv(bike_csv)
bike_df.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
0,18222186,15060,2017-01-19T17:05:00.000,2017-01-19T21:16:00.000,3031.0,34.044701,-118.252441,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9518671,77160,2016-10-09T14:37:00.000,2016-10-10T12:03:00.000,NaN,NaN,NaN,3000.0,NaN,NaN,...,One Way,Monthly Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20444932,86400,2017-02-18T10:15:00.000,2017-02-20T15:20:00.000,3026.0,34.063179,-118.245880,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20905031,18840,2017-02-27T12:26:00.000,2017-02-27T17:40:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,One Way,Walk-up,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21031476,86400,2017-02-27T20:26:00.000,2017-03-01T09:49:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Flex Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Looking at and cleaning the data

In [4]:
# Look at the size of the data
crimes_df.shape

(201764, 16)

In [5]:
# Look at the size of the data
bike_df.shape

(132427, 22)

In [6]:
# Check to see if any NaN's in crimes and how much
crimes_df["Zipcode"].isnull().sum()

0

In [9]:
# Check to see if any NaN's in crimes and how much
bike_df["Zip Codes"].isnull().sum()

33837

In [10]:
# Check to see if any NaN's in crimes and how much
bike_df["Starting Station Latitude"].isnull().sum()

48

In [11]:
# Check to see if any NaN's in crimes and how much
bike_df["Starting Station Longitude"].isnull().sum()

48

In [15]:
# Drop the NaN's in bikes and check to see how much data is left
bike_df_no_nans = bike_df.dropna(subset=["Zip Codes", "Starting Station Latitude", "Starting Station Longitude"])
bike_df_no_nans.shape

(98590, 22)

In [16]:
bike_df_no_nans.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Trip Route Category,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts
14,16883024,11100,2016-12-28T15:27:00.000,2016-12-28T18:32:00.000,3016.0,34.052898,-118.241562,NaN,NaN,NaN,...,One Way,Staff Annual,"{'longitude': '-118.241562', 'latitude': '34.0...",NaN,76.0,9.0,22728.0,NaN,NaN,NaN
32,15177219,13920,2016-12-05T10:43:00.000,2016-12-05T14:35:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.262733', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN
35,12227457,14580,2016-11-03T17:21:00.000,2016-11-03T21:24:00.000,3027.0,34.049980,-118.247162,3000.0,NaN,NaN,...,One Way,Flex Pass,"{'longitude': '-118.247162', 'latitude': '34.0...",NaN,76.0,9.0,23074.0,NaN,NaN,NaN
40,16726207,1320,2016-12-25T17:15:00.000,2016-12-25T17:37:00.000,3078.0,34.064281,-118.238937,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.238937', 'latitude': '34.0...",NaN,28.0,11.0,22728.0,NaN,NaN,NaN
43,13913164,11160,2016-11-19T15:39:00.000,2016-11-19T18:45:00.000,3025.0,34.032860,-118.268082,3000.0,NaN,NaN,...,One Way,Walk-up,"{'longitude': '-118.268082', 'latitude': '34.0...",NaN,76.0,9.0,23076.0,NaN,NaN,NaN


## Finding new Zip Codes

In [19]:
# Quick look at the Zip Codes and we can tell they are wrong. LA has zip codes in the 90000's
# Create new Zip Code Column
bike_df_no_nans.assign(New_Zip = "")

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip,New_Zip
14,16883024,11100,2016-12-28T15:27:00.000,2016-12-28T18:32:00.000,3016.0,34.052898,-118.241562,NaN,NaN,NaN,...,"{'longitude': '-118.241562', 'latitude': '34.0...",NaN,76.0,9.0,22728.0,NaN,NaN,NaN,,
32,15177219,13920,2016-12-05T10:43:00.000,2016-12-05T14:35:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,"{'longitude': '-118.262733', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN,,
35,12227457,14580,2016-11-03T17:21:00.000,2016-11-03T21:24:00.000,3027.0,34.049980,-118.247162,3000.0,NaN,NaN,...,"{'longitude': '-118.247162', 'latitude': '34.0...",NaN,76.0,9.0,23074.0,NaN,NaN,NaN,,
40,16726207,1320,2016-12-25T17:15:00.000,2016-12-25T17:37:00.000,3078.0,34.064281,-118.238937,3000.0,NaN,NaN,...,"{'longitude': '-118.238937', 'latitude': '34.0...",NaN,28.0,11.0,22728.0,NaN,NaN,NaN,,
43,13913164,11160,2016-11-19T15:39:00.000,2016-11-19T18:45:00.000,3025.0,34.032860,-118.268082,3000.0,NaN,NaN,...,"{'longitude': '-118.268082', 'latitude': '34.0...",NaN,76.0,9.0,23076.0,NaN,NaN,NaN,,
72,10990870,12060,2016-10-22T11:45:00.000,2016-10-22T15:06:00.000,3032.0,34.049889,-118.255882,3000.0,NaN,NaN,...,"{'longitude': '-118.255882', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN,,
92,14991982,180,2016-12-03T11:12:00.000,2016-12-03T11:15:00.000,3048.0,34.041691,-118.235352,NaN,NaN,NaN,...,"{'longitude': '-118.235352', 'latitude': '34.0...",NaN,28.0,9.0,23074.0,NaN,NaN,NaN,,
109,14792947,23100,2016-11-30T14:47:00.000,2016-11-30T21:12:00.000,3048.0,34.041691,-118.235352,3000.0,NaN,NaN,...,"{'longitude': '-118.235352', 'latitude': '34.0...",NaN,28.0,9.0,23074.0,NaN,NaN,NaN,,
187,16189278,360,2016-12-17T16:36:00.000,2016-12-17T16:42:00.000,3023.0,34.050911,-118.240967,3000.0,NaN,NaN,...,"{'longitude': '-118.240967', 'latitude': '34.0...",NaN,28.0,9.0,22728.0,NaN,NaN,NaN,,
210,16938072,19380,2016-12-29T14:03:00.000,2016-12-29T19:26:00.000,3042.0,34.049301,-118.238808,NaN,NaN,NaN,...,"{'longitude': '-118.238808', 'latitude': '34.0...",NaN,28.0,9.0,22728.0,NaN,NaN,NaN,,


In [71]:
for index, row in bike_df_no_nans.iterrows():
    
    lat = row["Starting Station Latitude"]
    long = row["Starting Station Longitude"]


    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
                  'latlng={lat},{long}&key={gkey}').format(lat=lat, long=long, gkey=gkey)
    response = requests.get(target_url).json()
    
    try:
        bike_df_no_nans.loc[index, 'New Zip'] = int(response['results'][0]['address_components'][7]['long_name'])
#         print(f"Adding {response['results'][0]['address_components'][7]['long_name']}")
    except:
        bike_df_no_nans.loc[index, 'New Zip'] = np.nan
        print(f"Couldn't append row {index}")
print(f'Finished retrieving zip code data')

Adding 90012
Couldn't append row 32
Adding 90013
Adding 90012
Adding 90015
Adding 90071
Adding 90013
Adding 90013
Adding 90012
Adding 90012
Couldn't append row 216
Adding 90017
Adding 90014
Adding 90014
Adding 90012
Adding 90014
Couldn't append row 343
Adding 90012
Adding 90014
Adding 90013
Finished retrieving zip code data


In [72]:
# See how many 'New Zip' NaN's there are
bike_df_no_nans["New Zip"].isnull().sum()

3

In [76]:
bike_df_no_nans.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip
14,16883024,11100,2016-12-28T15:27:00.000,2016-12-28T18:32:00.000,3016.0,34.052898,-118.241562,NaN,NaN,NaN,...,Staff Annual,"{'longitude': '-118.241562', 'latitude': '34.0...",NaN,76.0,9.0,22728.0,NaN,NaN,NaN,90012
32,15177219,13920,2016-12-05T10:43:00.000,2016-12-05T14:35:00.000,3008.0,34.046612,-118.262733,3000.0,NaN,NaN,...,Walk-up,"{'longitude': '-118.262733', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN,NaN
35,12227457,14580,2016-11-03T17:21:00.000,2016-11-03T21:24:00.000,3027.0,34.049980,-118.247162,3000.0,NaN,NaN,...,Flex Pass,"{'longitude': '-118.247162', 'latitude': '34.0...",NaN,76.0,9.0,23074.0,NaN,NaN,NaN,90013
40,16726207,1320,2016-12-25T17:15:00.000,2016-12-25T17:37:00.000,3078.0,34.064281,-118.238937,3000.0,NaN,NaN,...,Walk-up,"{'longitude': '-118.238937', 'latitude': '34.0...",NaN,28.0,11.0,22728.0,NaN,NaN,NaN,90012
43,13913164,11160,2016-11-19T15:39:00.000,2016-11-19T18:45:00.000,3025.0,34.032860,-118.268082,3000.0,NaN,NaN,...,Walk-up,"{'longitude': '-118.268082', 'latitude': '34.0...",NaN,76.0,9.0,23076.0,NaN,NaN,NaN,90015


In [81]:
# Drop the NaN's from dataframe
bike_df_no_nans_zip = bike_df_no_nans.dropna(subset=["New Zip"])
bike_df_no_nans_zip.head()

,Trip ID,Duration,Start Time,End Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Ending Station ID,Ending Station Latitude,Ending Station Longitude,...,Passholder Type,Starting Lat-Long,Ending Lat-Long,Neighborhood Councils (Certified),Council Districts,Zip Codes,LA Specific Plans,Precinct Boundaries,Census Tracts,New Zip
14,16883024,11100,2016-12-28T15:27:00.000,2016-12-28T18:32:00.000,3016.0,34.052898,-118.241562,NaN,NaN,NaN,...,Staff Annual,"{'longitude': '-118.241562', 'latitude': '34.0...",NaN,76.0,9.0,22728.0,NaN,NaN,NaN,90012
35,12227457,14580,2016-11-03T17:21:00.000,2016-11-03T21:24:00.000,3027.0,34.049980,-118.247162,3000.0,NaN,NaN,...,Flex Pass,"{'longitude': '-118.247162', 'latitude': '34.0...",NaN,76.0,9.0,23074.0,NaN,NaN,NaN,90013
40,16726207,1320,2016-12-25T17:15:00.000,2016-12-25T17:37:00.000,3078.0,34.064281,-118.238937,3000.0,NaN,NaN,...,Walk-up,"{'longitude': '-118.238937', 'latitude': '34.0...",NaN,28.0,11.0,22728.0,NaN,NaN,NaN,90012
43,13913164,11160,2016-11-19T15:39:00.000,2016-11-19T18:45:00.000,3025.0,34.032860,-118.268082,3000.0,NaN,NaN,...,Walk-up,"{'longitude': '-118.268082', 'latitude': '34.0...",NaN,76.0,9.0,23076.0,NaN,NaN,NaN,90015
72,10990870,12060,2016-10-22T11:45:00.000,2016-10-22T15:06:00.000,3032.0,34.049889,-118.255882,3000.0,NaN,NaN,...,Flex Pass,"{'longitude': '-118.255882', 'latitude': '34.0...",NaN,76.0,9.0,23078.0,NaN,NaN,NaN,90071


## Get the clean dataframe to be used

In [116]:
final_bike = bike_df_no_nans_zip[["New Zip", "Trip ID"]].copy()
final_bike.head()

,New Zip,Trip ID
14,90012,16883024
35,90013,12227457
40,90012,16726207
43,90015,13913164
72,90071,10990870


In [117]:
final_bike.shape

(98587, 2)

In [118]:
groupby_zip_bikes = final_bike.groupby("New Zip").count()
groupby_zip_bikes.head()

,Trip ID
New Zip,
90012,6
90013,4
90014,4
90015,1
90017,1


In [119]:
groupby_zip_bikes.reset_index(inplace=True)
groupby_zip_bikes

,New Zip,Trip ID
0,90012,6
1,90013,4
2,90014,4
3,90015,1
4,90017,1
5,90071,1
6,,98570


In [120]:
# Get only the columns that we need and make a new dataframe
new_crimes_df = crimes_df[["DR.NO", "Zipcode"]].copy()
new_crimes_df.head()

,DR.NO,Zipcode
0,132007717,90004
1,130608787,90068
2,131820260,90002
3,131817514,90002
4,130510483,90501


In [121]:
new_crimes_df.shape

(201764, 2)

In [122]:
groupby_zip_crimes = new_crimes_df.groupby("Zipcode").count()
groupby_zip_crimes.head()

,DR.NO
Zipcode,
90001,663
90002,2263
90003,6023
90004,2922
90005,1906


In [123]:
# Reset set the index
groupby_zip_crimes.reset_index(inplace=True)
groupby_zip_crimes.head()

,Zipcode,DR.NO
0,90001,663
1,90002,2263
2,90003,6023
3,90004,2922
4,90005,1906


In [ ]:
crimes_html = new_crimes_df.head(20).to_html(classes=["table-bordered", "table-striped", "table-hover"])
crimes_html